In [295]:
%%time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

pd.set_option('max_columns', 50)

# Marking out games by year
num_games = 22*9
end_of_2016 = 6369
end_of_2015 = 6161
end_of_2014 = 5954
end_of_2013 = 5747
end_of_2012 = 5540

# Which games to get - tweaked each time this is run
first_game = end_of_2012
before_last_game = end_of_2012 - num_games

pl = pd.DataFrame() # Player statistics
ma = pd.DataFrame() # Match attributes

# Two difference types of pages, normal stats and advanced stats
pages = [('', False), ('&advv=Y', True)] 

# Iterate over both types of pages, for the same match, normal stats and advanced stats
for page in pages:
    for i in range(first_game, before_last_game, -1): # Iterate over all matches starting at end of 2016 and working backwards
        soup = BeautifulSoup(requests.get(
                'http://www.footywire.com/afl/footy/ft_match_statistics?mid=%s%s'%(i,page[0])).text, 'lxml')
        
        # Don't scrap pages that are not found.
        if soup(text = 'Page Not Found'):
            print('No page %s%s'%(i,page[0]))
            continue
        
        #Find player statistics for this game for both teams
        data_tables = []
        for elem in soup(text='Player'):
            data_tables.append(elem.parent.parent.parent)
        match_attribs = [i]
        #Iterate over both teams
        rows = []
        for data_table in data_tables:
            above_team_title = data_table.parent.parent.parent.parent
            team_text = above_team_title.find_next('a').next_sibling # Text that has team name
            match_attribs.append(team_text.find_next('a').string) # Coach
            team = team_text[:team_text.index(' Match')] # Team
            match_attribs.append(team) 
            cols = [i, page[1], team]
            for tr in data_table.find_all('tr'): # Iterate over all players (inc header)
                for td in tr.find_all('td'):
                    cols.append(td.string)
                rows.append(cols)
                cols = [i, page[1], team]
        pl = pl.append(rows)

        # Only get game attributes once
        if page[0]:
            #Find and extract the game attributes
            round_stadium_attendance = soup.find(text = re.compile('Attendance:'))
            date_and_time = round_stadium_attendance.next_element.next_element.contents[0].string
            date_and_time = date_and_time.split(', ')
            match_attribs.append(date_and_time[1]) # Date
            match_attribs.append(date_and_time[2]) # Time
            round_stadium_attendance = round_stadium_attendance.split(', ')
            match_attribs.append(round_stadium_attendance[0].split(' ')[1]) # Round
            match_attribs.append(round_stadium_attendance[1]) # Stadium
            match_attribs.append(round_stadium_attendance[2].split(' ')[1]) # Attendance

            # game attributes - Quarter by quarter scores
            score_table = soup.find('table', attrs = {'id' : 'matchscoretable'})
            for tr in score_table.find_all('tr'):
                for th in tr:
                    val = th.string
                    if len(val.strip()) != 0:
                        match_attribs.append(val)
            
            # Get browlow votes (if they exist yet), 3,2,1 top to bottom
            brownlow = soup(text='Brownlow Votes:')
            if brownlow:
                def get_brownlow_team_and_player(a):
                    t = a[a.index('pp-')+3:a.index('--')]
                    t = t.replace('-', ' ')
                    t = t.title()
                    p = a[a.index('--')+2:]
                    p = p.replace('-', ' ')
                    p = p.title()
                    return [t, p]
                brownlow = brownlow[0]
                brownlow = brownlow.find_next('a')
                match_attribs += get_brownlow_team_and_player(brownlow.attrs['href'])
                brownlow = brownlow.find_next('a')
                match_attribs += get_brownlow_team_and_player(brownlow.attrs['href'])
                brownlow = brownlow.find_next('a')
                match_attribs += get_brownlow_team_and_player(brownlow.attrs['href'])
                
            ma = ma.append([match_attribs])
               
pl.to_excel('players_%s_to_%s.xlsx'%(first_game, before_last_game+1), header = False, index = False)
ma.to_excel('matches_%s_to_%s.xlsx'%(first_game, before_last_game+1), header = False, index = False)

CPU times: user 1min 59s, sys: 27 s, total: 2min 26s
Wall time: 7min 8s


In [309]:
# Combine files
match_ids = ['5540_to_5343', '5747_to_5550', '5954_to_5757', '6161_to_5964']

ma = pd.DataFrame()
pl = pd.DataFrame()
things = []

for i in match_ids:
    df = pd.read_excel('matches_%s.xlsx'%i, header = None, index = False)
    ma = ma.append(df)
    df = pd.read_excel('players_%s.xlsx'%i, header = None, index = False)
    things.append(df)
    pl = pl.append(df)
    
ma.to_excel('matches_unclean.xlsx', header = False, index = False)
pl.to_excel('players_unclean.xlsx', header = False, index = False)


In [483]:
# Open data to clean it
players = pd.read_excel('players_unclean.xlsx', header = None, index = False)
matches = pd.read_excel('matches_unclean.xlsx', header = None, index = False)

# Check the columns were all the same for players table
df = players
df = df[df[3] == 'Player']
for i in range(16):
    print df[i].value_counts()
    
# Note: Expect to see geelong and adelaide with one less game, due to cancelled game in 2015 for phil walsh
# Also all stats, will have the same number.

6143    4
5884    4
5880    4
5878    4
5876    4
5874    4
5872    4
5870    4
5868    4
5866    4
5864    4
5862    4
5860    4
5858    4
5856    4
5854    4
5852    4
5850    4
5848    4
5846    4
5844    4
5842    4
5840    4
5882    4
5886    4
5934    4
5888    4
5930    4
5928    4
5926    4
       ..
5533    4
5531    4
5529    4
5527    4
5525    4
5577    4
5579    4
5581    4
5605    4
5623    4
5621    4
5619    4
5617    4
5615    4
5613    4
5611    4
5609    4
5607    4
5603    4
5583    4
5601    4
5599    4
5597    4
5595    4
5593    4
5591    4
5589    4
5587    4
5585    4
6145    4
Name: 0, dtype: int64
True     1582
False    1582
Name: 1, dtype: int64
Carlton             176
Port Adelaide       176
Essendon            176
Fremantle           176
Brisbane            176
Sydney              176
Hawthorn            176
West Coast          176
Richmond            176
GWS                 176
Western Bulldogs    176
St Kilda            176
Gold Coast          176
Melbou

In [313]:
df = matches
df.info()
df[0].value_counts()
# Expect to see 791 games = 22 rounds * 9 games/round - 1 game (phil walsh's death)
# 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 34 columns):
0     791 non-null int64
1     791 non-null object
2     791 non-null object
3     791 non-null object
4     791 non-null object
5     791 non-null object
6     791 non-null object
7     791 non-null int64
8     791 non-null object
9     791 non-null int64
10    791 non-null object
11    791 non-null object
12    791 non-null object
13    791 non-null object
14    791 non-null object
15    791 non-null object
16    791 non-null object
17    791 non-null float64
18    791 non-null float64
19    791 non-null float64
20    791 non-null float64
21    791 non-null int64
22    791 non-null object
23    791 non-null float64
24    791 non-null float64
25    791 non-null float64
26    791 non-null float64
27    791 non-null int64
28    791 non-null object
29    791 non-null object
30    791 non-null object
31    791 non-null object
32    791 non-null object
33    791 non-null object
dtypes: 

6143    1
5604    1
5602    1
5601    1
5600    1
5599    1
5598    1
5597    1
5596    1
5595    1
5594    1
5593    1
5592    1
5591    1
5590    1
5589    1
5588    1
5587    1
5586    1
5585    1
5584    1
5583    1
5582    1
5603    1
5605    1
5629    1
5606    1
5627    1
5626    1
5625    1
       ..
5843    1
5842    1
5841    1
5840    1
5839    1
5860    1
5861    1
5862    1
5874    1
5883    1
5882    1
5881    1
5880    1
5879    1
5878    1
5877    1
5876    1
5875    1
5873    1
5863    1
5872    1
5871    1
5870    1
5869    1
5868    1
5867    1
5866    1
5865    1
5864    1
6144    1
Name: 0, dtype: int64

In [328]:

df = players

# Get the names of the normal players stats, and advanced player stats
norm_stat_names = list(df[(df[1] == False)&(df[3]=='Player')].iloc[0][4:])
adv_stat_names = list(df[(df[1] == True)&(df[3]=='Player')].iloc[0][4:])

df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,5540,False,GWS,Player,K,HB,D,M,G,B,T,HO,I50,FF,FA,AF,SC
1,5540,False,GWS,Toby Greene,22,16,38,2,1,2,2,0,6,2,3,113,100
2,5540,False,GWS,Callan Ward,14,16,30,2,1,1,3,2,10,3,0,104,115
3,5540,False,GWS,Dom Tyson,13,13,26,5,1,0,2,0,3,0,2,88,95
4,5540,False,GWS,Tom Scully,12,13,25,2,0,1,1,0,4,0,2,67,91
5,5540,False,GWS,Taylor Adams,10,13,23,1,0,0,4,0,2,3,1,75,82
6,5540,False,GWS,Adam Treloar,12,11,23,6,1,0,3,0,5,0,0,94,87
7,5540,False,GWS,Luke Power,8,11,19,2,0,0,0,0,1,2,1,51,59
8,5540,False,GWS,Rhys Palmer,8,11,19,3,3,1,2,2,2,0,0,84,97
9,5540,False,GWS,Adam Kennedy,7,10,17,5,0,0,1,0,1,1,0,61,56


In [415]:
# Restructure players table to have all players as rows
df = players.copy()

df = df[df[3]!='Player']
for i in range(4,17):
    df[i] = df[i].astype(float)

start_cols = ['id', 'Team', 'Player']
df1 = df[df[1]==False] # norm_stat_names
df2 = df[df[1]==True] # adv_stat_names
df1.drop(1, axis=1, inplace=True) 
df2.drop(1, axis=1, inplace=True)

players2 = df1.merge(df2, left_on=[0,2,3], right_on=[0,2,3])
players2

/Users/justinbarton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/justinbarton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,0,2,3,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y
0,5540,GWS,Toby Greene,22.0,16.0,38.0,2.0,1.0,2.0,2.0,0.0,6.0,2.0,3.0,113.0,100.0,17.0,23.0,19.0,50.0,11.0,0.0,1.0,0.0,8.0,0.0,1.0,0.0,110.0
1,5540,GWS,Callan Ward,14.0,16.0,30.0,2.0,1.0,1.0,3.0,2.0,10.0,3.0,0.0,104.0,115.0,16.0,14.0,20.0,66.7,2.0,0.0,2.0,0.0,7.0,1.0,0.0,0.0,89.0
2,5540,GWS,Dom Tyson,13.0,13.0,26.0,5.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,88.0,95.0,9.0,17.0,19.0,73.1,6.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,89.0
3,5540,GWS,Tom Scully,12.0,13.0,25.0,2.0,0.0,1.0,1.0,0.0,4.0,0.0,2.0,67.0,91.0,9.0,17.0,19.0,76.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,89.0
4,5540,GWS,Taylor Adams,10.0,13.0,23.0,1.0,0.0,0.0,4.0,0.0,2.0,3.0,1.0,75.0,82.0,11.0,12.0,15.0,65.2,1.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,78.0
5,5540,GWS,Adam Treloar,12.0,11.0,23.0,6.0,1.0,0.0,3.0,0.0,5.0,0.0,0.0,94.0,87.0,4.0,18.0,17.0,73.9,0.0,0.0,1.0,0.0,2.0,1.0,1.0,3.0,86.0
6,5540,GWS,Luke Power,8.0,11.0,19.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,51.0,59.0,12.0,8.0,15.0,78.9,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,90.0
7,5540,GWS,Rhys Palmer,8.0,11.0,19.0,3.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,84.0,97.0,8.0,11.0,14.0,73.7,0.0,0.0,0.0,3.0,2.0,1.0,2.0,0.0,118.0
8,5540,GWS,Adam Kennedy,7.0,10.0,17.0,5.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,61.0,56.0,3.0,13.0,15.0,88.2,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,86.0
9,5540,GWS,Curtly Hampton,13.0,4.0,17.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,54.0,57.0,4.0,11.0,10.0,58.8,3.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,83.0


In [472]:
# Check the team names in players table
df = players2
players_team_names = sorted(list(df[2].value_counts().index))
df = matches
team_names = [[0] + players_team_names]
for i in [2, 4, 28, 30, 32]:    
    team_names.append([i]+sorted(list(df[i].value_counts().index)))

df = pd.DataFrame(team_names)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,Adelaide,Brisbane,Carlton,Collingwood,Essendon,Fremantle,GWS,Geelong,Gold Coast,Hawthorn,Melbourne,North Melbourne,Port Adelaide,Richmond,St Kilda,Sydney,West Coast,Western Bulldogs
1,2,Adelaide,Brisbane,Carlton,Collingwood,Essendon,Fremantle,GWS,Geelong,Gold Coast,Hawthorn,Melbourne,North Melbourne,Port Adelaide,Richmond,St Kilda,Sydney,West Coast,Western Bulldogs
2,4,Adelaide,Brisbane,Carlton,Collingwood,Essendon,Fremantle,GWS,Geelong,Gold Coast,Hawthorn,Melbourne,North Melbourne,Port Adelaide,Richmond,St Kilda,Sydney,West Coast,Western Bulldogs
3,28,Adelaide Crows,Brisbane Lions,Carlton Blues,Collingwood Magpies,Essendon Bombers,Fremantle Dockers,Geelong Cats,Gold Coast Suns,Greater Western Sydney Giants,Hawthorn Hawks,Kangaroos,Melbourne Demons,Port Adelaide Power,Richmond Tigers,St Kilda Saints,Sydney Swans,West Coast Eagles,Western Bulldogs
4,30,Adelaide Crows,Brisbane Lions,Carlton Blues,Collingwood Magpies,Essendon Bombers,Fremantle Dockers,Geelong Cats,Gold Coast Suns,Greater Western Sydney Giants,Hawthorn Hawks,Kangaroos,Melbourne Demons,Port Adelaide Power,Richmond Tigers,St Kilda Saints,Sydney Swans,West Coast Eagles,Western Bulldogs
5,32,Adelaide Crows,Brisbane Lions,Carlton Blues,Collingwood Magpies,Essendon Bombers,Fremantle Dockers,Geelong Cats,Gold Coast Suns,Greater Western Sydney Giants,Hawthorn Hawks,Kangaroos,Melbourne Demons,Port Adelaide Power,Richmond Tigers,St Kilda Saints,Sydney Swans,West Coast Eagles,Western Bulldogs


In [491]:
# Correct the team names
team_map = {'Adelaide Crows':'Adelaide', 'Brisbane Lions':'Brisbane', 'Carlton Blues':'Carlton',
            'Collingwood Magpies':'Collingwood', 'Essendon Bombers':'Essendon','Fremantle Dockers':'Fremantle',
            'Greater Western Sydney Giants':'GWS', 'Geelong Cats':'Geelong','Gold Coast Suns':'Gold Coast',
            'Hawthorn Hawks':'Hawthorn', 'Kangaroos':'North Melbourne', 'Melbourne Demons':'Melbourne', 
            'Port Adelaide Power':'Port Adelaide', 'Richmond Tigers':'Richmond', 'St Kilda Saints':'St Kilda',
            'Sydney Swans':'Sydney','West Coast Eagles':'West Coast', 'Western Bulldogs':'Western Bulldogs'}

df = matches.copy()
for i in [28, 30, 32]:
    df[i] = df[i].map(lambda x: team_map[str(x)])
    print sorted(list(df[i].value_counts().index))
    
matches2 = df

['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'GWS', 'Geelong', 'Gold Coast', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']
['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'GWS', 'Geelong', 'Gold Coast', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']
['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'GWS', 'Geelong', 'Gold Coast', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']


In [529]:
# Make sure brownlow votes names match in each table
df = players2.merge(matches2[[28, 29]], left_on = [2,3], right_on = [28, 29], how = 'right')
df = df[df[0].isnull()]
mismatches = df[[2,3]].drop_duplicates()
df = players2.merge(matches2[[30, 31]], left_on = [2,3], right_on = [30, 31], how = 'right')
df = df[df[0].isnull()]
mismatches = mismatches.append(df[[2,3]]).drop_duplicates()
df = players2.merge(matches2[[32, 33]], left_on = [2,3], right_on = [32, 33], how = 'right')
df = df[df[0].isnull()]
mismatches = mismatches.append(df[[2,3]]).drop_duplicates()
mismatches

,2,3
50562,North Melbourne,Scott Thompson 1
50564,Carlton,Brock Mclean
50567,Gold Coast,Thomas Lynch
50569,Sydney,Josh P Kennedy
50588,St Kilda,Beau Wilkes
50589,Sydney,Ryan O Keefe
50592,Sydney,Jarrad Mcveigh
50595,Sydney,Kieran Jack
50601,Western Bulldogs,Brian Harris
50602,Port Adelaide,Bradley Ebert


In [548]:
# Find the closest matches for each using difflib
import difflib
df = mismatches
df['name'] = df[3].map(lambda x: difflib.get_close_matches(x, players2[3].drop_duplicates(), cutoff = 0.7, n = 4))
mismatches2 = df
df

,2,3,name
50562,North Melbourne,Scott Thompson 1,[Scott Thompson]
50564,Carlton,Brock Mclean,"[Brock McLean, Broc McCauley]"
50567,Gold Coast,Thomas Lynch,"[Tom Lynch, Thomas Couch, Thomas Lee]"
50569,Sydney,Josh P Kennedy,"[Josh P. Kennedy, Joshua Kennedy]"
50588,St Kilda,Beau Wilkes,[Beau Waters]
50589,Sydney,Ryan O Keefe,"[Ryan O'Keefe, Rhys O'Keeffe]"
50592,Sydney,Jarrad Mcveigh,[Jarrad McVeigh]
50595,Sydney,Kieran Jack,[Kieren Jack]
50601,Western Bulldogs,Brian Harris,[]
50602,Port Adelaide,Bradley Ebert,"[Brad Ebert, Brett Ebert, Bradley Hartman]"


In [573]:
df = mismatches2.copy()
# Fix some special cases.
df.loc[df[3]=='Brian Harris', 'name'] = [['Brian Lake']]
df.loc[df[3]=='Heritier O Brien', 'name'] = [['Heritier Lumumba']]
df.loc[df[3]=='Beau Wilkes', 'name'] = [['Beau Maister']]

#Take the first match from each of 'name'
df['name'] = df['name'].map(lambda x: x[0])
df.columns = ['team', 'old', 'new']
player_map = df
df

,team,old,new
50562,North Melbourne,Scott Thompson 1,Scott Thompson
50564,Carlton,Brock Mclean,Brock McLean
50567,Gold Coast,Thomas Lynch,Tom Lynch
50569,Sydney,Josh P Kennedy,Josh P. Kennedy
50588,St Kilda,Beau Wilkes,Beau Maister
50589,Sydney,Ryan O Keefe,Ryan O'Keefe
50592,Sydney,Jarrad Mcveigh,Jarrad McVeigh
50595,Sydney,Kieran Jack,Kieren Jack
50601,Western Bulldogs,Brian Harris,Brian Lake
50602,Port Adelaide,Bradley Ebert,Brad Ebert


In [605]:
# Function to map names
def map_names(old_name):
    result = player_map.loc[player_map['old'] == old_name, 'new']
    if len(result) > 0:
        return result.iloc[0]
    else:
        return old_name

In [607]:
# Apply player map to names in matches2
df = matches2.copy()
bl_names = [29, 31, 33]
for i in bl_names:
    df[i] = df[i].map(lambda x: map_names(x))
matches3 = df
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,5540,Kevin Sheedy,GWS,Brad Scott,North Melbourne,1st September 2012,4:40 PM AEST,23,Spotless Stadium,6696,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,GWS,3.3,4.50,7.80,11.11,77,North Melbourne,3.2,7.40,12.50,16.90,105,North Melbourne,Scott Thompson,GWS,Callan Ward,GWS,Toby Greene
1,5539,Ross Lyon,Fremantle,Mark Neeld,Melbourne,1st September 2012,5:40 PM AWST,23,Domain Stadium,32687,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Fremantle,3.5,7.70,11.14,14.17,101,Melbourne,1.3,2.50,4.80,5.10,40,Fremantle,Nathan Fyfe,Fremantle,David Mundy,Fremantle,Aaron Sandilands
2,5538,Damien Hardwick,Richmond,Garry Hocking,Port Adelaide,2nd September 2012,3:15 PM AEST,23,MCG,27893,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Richmond,5.1,6.60,11.80,16.10,106,Port Adelaide,2.5,6.70,12.80,16.10,106,Richmond,Trent Cotchin,Port Adelaide,Brad Ebert,Richmond,Jack Riewoldt
3,5537,Chris Scott,Geelong,John Longmire,Sydney,1st September 2012,1:45 PM AEST,23,Simonds Stadium,20045,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Geelong,4.3,7.50,12.70,17.10,112,Sydney,4.5,5.80,7.12,11.12,78,Geelong,Joel Selwood,Geelong,Harry Taylor,Sydney,Shane Mumford
4,5536,Alastair Clarkson,Hawthorn,John Worsfold,West Coast,31st August 2012,7:50 PM AEST,23,MCG,50023,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Hawthorn,7.1,10.60,11.10,14.11,95,West Coast,1.3,4.40,5.70,10.10,70,Hawthorn,Sam Mitchell,West Coast,Scott Selwood,Hawthorn,Lance Franklin
5,5535,Michael Voss,Brisbane,Brendan McCartney,Western Bulldogs,2nd September 2012,4:40 PM AEST,23,Gabba,18289,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Brisbane,1.4,9.70,14.12,19.14,128,Western Bulldogs,0.3,2.50,4.60,9.70,61,Brisbane,Daniel Rich,Brisbane,Jack Redden,Brisbane,Claye Beams
6,5534,Brett Ratten,Carlton,Scott Watters,St Kilda,2nd September 2012,1:10 PM AEST,23,Etihad Stadium,31393,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Carlton,4.5,7.70,10.15,12.19,91,St Kilda,4.1,8.40,10.60,16.10,106,Carlton,Brock McLean,St Kilda,David Armitage,St Kilda,Jason Gram
7,5533,Brenton Sanderson,Adelaide,Guy McKenna,Gold Coast,1st September 2012,1:40 PM ACST,23,AAMI Stadium,35463,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Adelaide,3.6,9.13,15.18,22.21,153,Gold Coast,1.3,5.60,7.60,9.80,62,Adelaide,Patrick Dangerfield,Adelaide,Scott Thompson,Adelaide,Ian Callinan
8,5532,James Hird,Essendon,Nathan Buckley,Collingwood,1st September 2012,7:40 PM AEST,23,MCG,56491,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Essendon,4.3,7.30,9.70,10.12,72,Collingwood,5.5,7.11,10.17,14.20,104,Collingwood,Dane Swan,Collingwood,Travis Cloke,Collingwood,Scott Pendlebury
9,5531,Mark Neeld,Melbourne,Brenton Sanderson,Adelaide,26th August 2012,4:40 PM AEST,22,MCG,18450,Team\n,Q1\n,Q2\n,Q3\n,Q4\n,Final\n,Melbourne,3.4,5.70,8.10,11.15,81,Adelaide,5.3,10.80,20.12,22.18,150,Adelaide,Taylor Walker,Adelaide,Patrick Dangerfield,Melbourne,Colin Sylvia


In [637]:
# Strip extraneous whitespace for each player and teamname
# IMPORTANT - without this, names won't match
df = matches3.copy()
for i in range(28,34):
    df[i] = df[i].map(lambda x: x.strip())
matches4 = df
df = players2.copy()
for i in [2,3]:
    df[i] = df[i].map(lambda x: x.strip())
players3 = df

In [656]:
# Check each game has 3 brownlow winners
df = players2.merge(matches4[[0, 28, 29]], left_on = [0, 2, 3], right_on = [0, 28, 29], how = 'left')
df = df.merge(matches4[[0, 30, 31]], left_on = [0, 2, 3], right_on = [0, 30, 31], how = 'left')
df = df.merge(matches4[[0, 32, 33]], left_on = [0, 2, 3], right_on = [0, 32, 33], how = 'left')
players3 = df
df = df[df[28].notnull()|df[30].notnull()|df[32].notnull()]
df[0].value_counts()

6143    3
5884    3
5880    3
5878    3
5876    3
5874    3
5872    3
5870    3
5868    3
5866    3
5864    3
5862    3
5860    3
5858    3
5856    3
5854    3
5852    3
5850    3
5848    3
5846    3
5844    3
5842    3
5840    3
5882    3
5886    3
5934    3
5888    3
5930    3
5928    3
5926    3
       ..
5533    3
5531    3
5529    3
5527    3
5525    3
5577    3
5579    3
5581    3
5605    3
5623    3
5621    3
5619    3
5617    3
5615    3
5613    3
5611    3
5609    3
5607    3
5603    3
5583    3
5601    3
5599    3
5597    3
5595    3
5593    3
5591    3
5589    3
5587    3
5585    3
6145    3
Name: 0, dtype: int64

In [658]:
# Remove extraenous cells from player table
df = players3.copy()
df = df.drop([28,30,32], axis=1)
for i in [29, 31, 33]:
    df.loc[df[i].notnull(), i] = 1.0
players4 = df
df

,0,2,3,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,29,31,33
0,5540,GWS,Toby Greene,22.0,16.0,38.0,2.0,1.0,2.0,2.0,0.0,6.0,2.0,3.0,113.0,100.0,17.0,23.0,19.0,50.0,11.0,0.0,1.0,0.0,8.0,0.0,1.0,0.0,110.0,NaN,NaN,1
1,5540,GWS,Callan Ward,14.0,16.0,30.0,2.0,1.0,1.0,3.0,2.0,10.0,3.0,0.0,104.0,115.0,16.0,14.0,20.0,66.7,2.0,0.0,2.0,0.0,7.0,1.0,0.0,0.0,89.0,NaN,1,NaN
2,5540,GWS,Dom Tyson,13.0,13.0,26.0,5.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,88.0,95.0,9.0,17.0,19.0,73.1,6.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,89.0,NaN,NaN,NaN
3,5540,GWS,Tom Scully,12.0,13.0,25.0,2.0,0.0,1.0,1.0,0.0,4.0,0.0,2.0,67.0,91.0,9.0,17.0,19.0,76.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,89.0,NaN,NaN,NaN
4,5540,GWS,Taylor Adams,10.0,13.0,23.0,1.0,0.0,0.0,4.0,0.0,2.0,3.0,1.0,75.0,82.0,11.0,12.0,15.0,65.2,1.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,78.0,NaN,NaN,NaN
5,5540,GWS,Adam Treloar,12.0,11.0,23.0,6.0,1.0,0.0,3.0,0.0,5.0,0.0,0.0,94.0,87.0,4.0,18.0,17.0,73.9,0.0,0.0,1.0,0.0,2.0,1.0,1.0,3.0,86.0,NaN,NaN,NaN
6,5540,GWS,Luke Power,8.0,11.0,19.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,51.0,59.0,12.0,8.0,15.0,78.9,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,90.0,NaN,NaN,NaN
7,5540,GWS,Rhys Palmer,8.0,11.0,19.0,3.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,84.0,97.0,8.0,11.0,14.0,73.7,0.0,0.0,0.0,3.0,2.0,1.0,2.0,0.0,118.0,NaN,NaN,NaN
8,5540,GWS,Adam Kennedy,7.0,10.0,17.0,5.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,61.0,56.0,3.0,13.0,15.0,88.2,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,86.0,NaN,NaN,NaN
9,5540,GWS,Curtly Hampton,13.0,4.0,17.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,54.0,57.0,4.0,11.0,10.0,58.8,3.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,83.0,NaN,NaN,NaN


In [666]:
# Check web scraping was correct, ie team names in correct order
# and other fields all exist on each page ('Team', 'Q1',....etc)
df = matches4
[len(df[df[2] != df[16]]), len(df[df[4] != df[22]]), df[10].value_counts(), df[11].value_counts(), 
 df[12].value_counts(), df[13].value_counts(), df[14].value_counts(), df[15].value_counts()]

[0, 0, Team\n    791
 Name: 10, dtype: int64, Q1\n    791
 Name: 11, dtype: int64, Q2\n    791
 Name: 12, dtype: int64, Q3\n    791
 Name: 13, dtype: int64, Q4\n    791
 Name: 14, dtype: int64, Final\n    791
 Name: 15, dtype: int64]

In [671]:
# Remove extraneous columns
df = matches4.copy()
df = df.drop([22,10,11,12,13,14,15,16], axis=1)
matches5 = df
df

,0,1,2,3,4,5,6,7,8,9,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33
0,5540,Kevin Sheedy,GWS,Brad Scott,North Melbourne,1st September 2012,4:40 PM AEST,23,Spotless Stadium,6696,3.3,4.50,7.80,11.11,77,3.2,7.40,12.50,16.90,105,North Melbourne,Scott Thompson,GWS,Callan Ward,GWS,Toby Greene
1,5539,Ross Lyon,Fremantle,Mark Neeld,Melbourne,1st September 2012,5:40 PM AWST,23,Domain Stadium,32687,3.5,7.70,11.14,14.17,101,1.3,2.50,4.80,5.10,40,Fremantle,Nathan Fyfe,Fremantle,David Mundy,Fremantle,Aaron Sandilands
2,5538,Damien Hardwick,Richmond,Garry Hocking,Port Adelaide,2nd September 2012,3:15 PM AEST,23,MCG,27893,5.1,6.60,11.80,16.10,106,2.5,6.70,12.80,16.10,106,Richmond,Trent Cotchin,Port Adelaide,Brad Ebert,Richmond,Jack Riewoldt
3,5537,Chris Scott,Geelong,John Longmire,Sydney,1st September 2012,1:45 PM AEST,23,Simonds Stadium,20045,4.3,7.50,12.70,17.10,112,4.5,5.80,7.12,11.12,78,Geelong,Joel Selwood,Geelong,Harry Taylor,Sydney,Shane Mumford
4,5536,Alastair Clarkson,Hawthorn,John Worsfold,West Coast,31st August 2012,7:50 PM AEST,23,MCG,50023,7.1,10.60,11.10,14.11,95,1.3,4.40,5.70,10.10,70,Hawthorn,Sam Mitchell,West Coast,Scott Selwood,Hawthorn,Lance Franklin
5,5535,Michael Voss,Brisbane,Brendan McCartney,Western Bulldogs,2nd September 2012,4:40 PM AEST,23,Gabba,18289,1.4,9.70,14.12,19.14,128,0.3,2.50,4.60,9.70,61,Brisbane,Daniel Rich,Brisbane,Jack Redden,Brisbane,Claye Beams
6,5534,Brett Ratten,Carlton,Scott Watters,St Kilda,2nd September 2012,1:10 PM AEST,23,Etihad Stadium,31393,4.5,7.70,10.15,12.19,91,4.1,8.40,10.60,16.10,106,Carlton,Brock McLean,St Kilda,David Armitage,St Kilda,Jason Gram
7,5533,Brenton Sanderson,Adelaide,Guy McKenna,Gold Coast,1st September 2012,1:40 PM ACST,23,AAMI Stadium,35463,3.6,9.13,15.18,22.21,153,1.3,5.60,7.60,9.80,62,Adelaide,Patrick Dangerfield,Adelaide,Scott Thompson,Adelaide,Ian Callinan
8,5532,James Hird,Essendon,Nathan Buckley,Collingwood,1st September 2012,7:40 PM AEST,23,MCG,56491,4.3,7.30,9.70,10.12,72,5.5,7.11,10.17,14.20,104,Collingwood,Dane Swan,Collingwood,Travis Cloke,Collingwood,Scott Pendlebury
9,5531,Mark Neeld,Melbourne,Brenton Sanderson,Adelaide,26th August 2012,4:40 PM AEST,22,MCG,18450,3.4,5.70,8.10,11.15,81,5.3,10.80,20.12,22.18,150,Adelaide,Taylor Walker,Adelaide,Patrick Dangerfield,Melbourne,Colin Sylvia


In [674]:
# Correct matches column names and save to excel
df = matches5
columns = ['id', 'Coach1','Team1','Coach2','Team2','Date','Time','Round','Stadium', 'Attendance', 'Team1Q1', 'Team1Q2',
          'Team1Q3','Team1Q4','Team1Score', 'Team2Q1', 'Team2Q2', 'Team2Q3', 'Team2Q4', 'Team2Score',
          'Team3V','Player3V','Team2V', 'Player2V','Team1V','Player1V']
df.columns = columns
df.to_excel('matches.xlsx', index = False)

In [679]:
# Correct players column names and save to excel
df = players4
columns = ['id', 'Team', 'Player'] + norm_stat_names + adv_stat_names + [3,2,1]
df.columns = columns
df.to_excel('players.xlsx', index = False)